<a href="https://colab.research.google.com/github/olonok69/LLM_Notebooks/blob/main/llms/LLM_Hallucinations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hallucinations

What are AI hallucinations?
AI hallucinations are incorrect or misleading results that AI models generate. These errors can be caused by a variety of factors, including insufficient training data, incorrect assumptions made by the model, or biases in the data used to train the model. AI hallucinations can be a problem for AI systems that are used to make important decisions, such as medical diagnoses or financial trading.


An AI hallucination is when an AI model generates incorrect information but presents it as if it were a fact. Why would it do that? AI tools like ChatGPT are trained to predict strings of words that best match your query. They lack the reasoning, however, to apply logic or consider any factual inconsistencies they're spitting out


# What causes AI hallucinations?
AI hallucinations can occur for several reasons, including:

- Insufficient, outdated, or low-quality training data. An AI model is only as good as the data it's trained on. If the AI tool doesn't understand your prompt or doesn't have sufficient information, it'll rely on the limited dataset it's been trained on to generate a response—even if it's inaccurate.

- Overfitting. When an AI model is trained on a limited dataset, it may memorize the inputs and appropriate outputs. This leaves it unable to effectively generalize new data, resulting in AI hallucinations.

- Use of idioms or slang expressions. If a prompt contains an idiom or slang expression that the AI model hasn't been trained on, it may lead to nonsensical outputs.

- Adversarial attacks. Prompts that are deliberately designed to confuse the AI can cause it to produce AI hallucinations.

https://cloud.google.com/discover/what-are-ai-hallucinations

https://kili-technology.com/large-language-models-llms/understanding-llm-hallucinations-and-how-to-mitigate-them

# Mitigation

1. Limit the possible outcomes
2. Pack in relevant data and sources unique to you
3. Create a data template for the model to follow
4. Give the AI a specific role—and tell it not to lie
5. Tell it what you want—and what you don't want
6. Experiment with the temperature

In [1]:
! pip install langchain langchain-community tiktoken -q
! pip install -U accelerate -q
! pip install -U unstructured numpy -q
! pip install openai chromadb beautifulsoup4 -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.1/269.1 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 72.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 94.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

# Mistral 7B

In [37]:
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", pad_token_id = 2)

messages = [
    {"role": "user", "content": "How many 'm's are in the word 'Weather'?"}
]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [38]:
encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

In [39]:
model_inputs = encodeds.to(device)
model = model.to(device)

In [40]:
generated_ids = model.generate(model_inputs, max_new_tokens=4000, do_sample=True, top_p=1,
    top_k=40,
    temperature=0.9 , pad_token_id =2)

In [41]:
import pprint
decoded = tokenizer.batch_decode(generated_ids)
pprint.pprint(decoded[0].split("[/INST]")[-1])

" The word 'Weather' contains two 'm's.</s>"


In [24]:
messages = [
    {"role": "user", "content": "How many 'm's are in the word 'Weather'?"},
    {"role": "assistant", "content": "'Weather' contains two 'm's. The first 'm' is in the letter 'M' in the word, \
    and the second 'm' is in the sound represented by the letter sequence 'th' in 'Weather', which is the sound of the letter 'm' in the middle of the word \
    'ether'. So, even though we don't see two 'm' letters when we write the word \
    'Weather', there are still two 'm' sounds in the word."},
    {"role": "user", "content": "Are you sure of your answer?"}
]


In [25]:
encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

In [26]:
model_inputs = encodeds.to(device)
generated_ids = model.generate(model_inputs, max_new_tokens=4000, do_sample=True, top_p=1,
    top_k=40,
    temperature=0.9 , pad_token_id =2)



In [27]:
decoded = tokenizer.batch_decode(generated_ids)
pprint.pprint(decoded[0].split("[/INST]")[-1])

(" Yes, I am sure of my answer that the word 'Weather' contains two 'm' "
 "sounds, even if it does not contain two 'm' letters when written. The 'th' "
 "in 'Weather' represents the sound of the 'm' in 'ether', making the total "
 "number of 'm' sounds in the word two.</s>")


## OPENAI

In [28]:
from langchain.llms import OpenAI
from langchain import LLMChain
from langchain.prompts.prompt import PromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
# Chat specific components
from langchain.memory import ConversationBufferMemory

In [29]:
from google.colab import userdata
openai_api_key = userdata.get('KEY_OPENAI')

In [30]:
from langchain_community.chat_models import ChatOpenAI

chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=1, openai_api_key=openai_api_key, max_tokens=250, model_kwargs ={"top_p":1, "frequency_penalty":0,"presence_penalty":0})

In [14]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | chat

In [31]:
text = """
How many 'm's are in the word 'Weather'?
"""
text2 = "Are you sure of your answer?"

In [32]:
response = chain.invoke(
    {
        "messages": [
            HumanMessage(
                content=f"{text}"
            ),

        ],
    }
)

In [33]:
pprint.pprint(response.content)

"There is 1 'm' in the word 'Weather'."


In [34]:

response = chat.invoke(
    [
        HumanMessage(
            content=f"{text}"
        ),
        AIMessage(content=response.content),
        HumanMessage(content=f"{text2}"),
    ]
)

In [35]:
pprint.pprint(response.content)

("My apologies for the previous response. There are actually no 'm's in the "
 "word 'Weather'. Thank you for pointing that out.")
